In [1]:
#imports
import nltk
nltk.data.path.append('./nltk_data/')
from nltk.corpus import brown, stopwords
import string
import numpy as np
import sys

#load and split words
corpus = brown.words()
stopwordsList = stopwords.words('english')
for i in range(len(string.punctuation)):
    stopwordsList.append(string.punctuation[i])
    stopwordsList.append((string.punctuation[i] + string.punctuation[i]))
corpus = [word.lower() for word in corpus if word.lower() not in stopwordsList]

# count words
wordsList = []
countList = []
for i in range(len(corpus)):
    if(corpus[i] in wordsList):
        countList[wordsList.index(corpus[i])] = countList[wordsList.index(corpus[i])] + 1
    else:
        wordsList.append(corpus[i])
        countList.append(1)
        
#make tuple list
finalList = []
for i in range(len(wordsList)):
    finalList.append((countList[i], wordsList[i]))
    
# order tuple list and split into our used words
finalList.sort(reverse=True)
vocab = finalList[:5000]
context = finalList[:1000]
contextWords = []
for i in range(1000):
    contextWords.append(context[i][1])
vocabWords = []
for i in range(5000):
    vocabWords.append(vocab[i][1])
totalContextWords = 0
for i in range(1000):
    totalContextWords = totalContextWords + context[i][0]
    
#calc window funct
nfunct = np.zeros((len(vocab), len(context)))
for i in range(len(vocab)):
    indices = [z for z, x in enumerate(corpus) if x == vocab[i][1]]
    for j in indices:
        if(j > 1 and corpus[(j-2)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j-2)])] = nfunct[i][contextWords.index(corpus[(j-2)])] + 1
        if(j > 0 and corpus[(j-1)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j-1)])] = nfunct[i][contextWords.index(corpus[(j-1)])] + 1
        if(j < (len(corpus)-2) and corpus[(j+2)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j+2)])] = nfunct[i][contextWords.index(corpus[(j+2)])] + 1
        if(j > (len(corpus)-1) and corpus[(j+1)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j+1)])] = nfunct[i][contextWords.index(corpus[(j+1)])] + 1

In [ ]:
#calc PCA embedding
from sklearn.decomposition import PCA

Cembedding = np.zeros((len(vocab), len(context)))
for i in range(len(vocab)):
    for j in range(len(context)):
        numer = (nfunct[i][j] / nfunct[i].sum())
        denom = (context[j][0] / totalContextWords)
        temp = (numer/denom)
        if(denom > numer):
            temp = 0
        else:
            temp = np.log(temp)
        Cembedding[i][(j)] = temp

pca = PCA(n_components=100)
finalEmbed = pca.fit_transform(Cembedding)

In [2]:
#calc SVD embedding
from sklearn.decomposition import TruncatedSVD

Cembedding = np.zeros((len(vocab), len(context)))
for i in range(len(vocab)):
    for j in range(len(context)):
        numer = (nfunct[i][j] / nfunct[i].sum())
        denom = (context[j][0] / totalContextWords)
        temp = (numer/denom)
        if(denom > numer):
            temp = 0
        else:
            temp = np.log(temp)
        Cembedding[i][(j)] = temp

svd = TruncatedSVD(n_components=100)
finalEmbed = svd.fit_transform(Cembedding)

In [3]:
#predict single words
searchWords = ["year", "communism", "autumn", "cigarette", "pulmonary", "mankind", "africa", "chicago", "revolution", "september",
"chemical", "detergent", "dictionary", "storm", "worship", "air", "university", "wife", "husband", "north", "wall", "literature", "doctor", "americans", "management"]
for asdf in range(len(searchWords)):
    word1 = vocabWords.index(searchWords[asdf])
    output = -1
    minDist = 10000000
    for i in range(5000):
        if(i == word1):
            continue
        dist = np.dot(finalEmbed[i], finalEmbed[word1])
        dist = dist / np.linalg.norm(finalEmbed[i])
        dist = dist / np.linalg.norm(finalEmbed[word1])
        dist = 1 - dist
        if(dist < minDist):
            minDist = dist
            output = i
    outputEuclid = -1
    minDist = 10000000
    for i in range(5000):
        if(i == word1):
            continue
        dist = finalEmbed[i] - finalEmbed[word1]
        dist = np.linalg.norm(dist)
        if(dist < minDist):
            minDist = dist
            outputEuclid = i
    outputManhat = -1
    minDist = 10000000
    for i in range(5000):
        if(i == word1):
            continue
        dist = 0
        for j in range(len(finalEmbed[i])):
            dist = dist + np.abs(finalEmbed[i][j] - finalEmbed[word1][j])
        if(dist < minDist):
            minDist = dist
            outputManhat = i
    print(" ",vocab[word1][1]," & ",vocab[outputEuclid][1]," & ",vocab[output][1]," & ",vocab[outputManhat][1],"\\\\")

  year  &  month  &  month  &  two \\
  communism  &  ethics  &  greatest  &  ethics \\
  autumn  &  seized  &  summer  &  helion \\
  cigarette  &  wiped  &  suddenly  &  wiped \\
  pulmonary  &  artery  &  artery  &  artery \\
  mankind  &  surrender  &  world  &  surrender \\
  africa  &  america  &  america  &  katanga \\
  chicago  &  portland  &  school  &  portland \\
  revolution  &  henrietta  &  henrietta  &  henrietta \\
  september  &  december  &  december  &  december \\
  chemical  &  drugs  &  drugs  &  drugs \\
  detergent  &  mustard  &  fabrics  &  fabrics \\
  dictionary  &  text  &  text  &  text \\
  storm  &  smelled  &  saturday  &  smelled \\
  worship  &  paying  &  community  &  paying \\
  air  &  little  &  head  &  suddenly \\
  university  &  college  &  college  &  college \\
  wife  &  old  &  told  &  old \\
  husband  &  wife  &  wife  &  mr. \\
  north  &  south  &  south  &  south \\
  wall  &  side  &  front  &  house \\
  literature  &  great  &  

In [ ]:
#cluster
import numpy as np
from sklearn.cluster import KMeans
from sklearn import datasets

target = []
for i in range(len(vocab)):
    target.append(vocab[i][1])
target = np.asarray(target)
estimator = KMeans(n_clusters=100)
estimator.fit(finalEmbed)

In [ ]:
#output clusters
for i in range(len(estimator.labels_)):
    if(estimator.labels_[i] == 2):
        print(target[i])

In [ ]:
for i in range(len(vocab)):
    print(i,": ",vocab[i])

In [4]:
from nltk.cluster.kmeans import KMeansClusterer
NUM_CLUSTERS = 100

kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=50)
assigned_clusters = kclusterer.cluster(finalEmbed, assign_clusters=True)

In [ ]:
zCount = 0
for i in range(len(vocab)):
    for j in range(len(context)):
        if(Cembedding[i][j] == 0):
            zCount = zCount + 1
print(zCount)

In [5]:
#output clusters
target = []
for i in range(len(vocab)):
    target.append(vocab[i][1])
for asdf in range(100):
    print("CLUST:",asdf,"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    for i in range(len(assigned_clusters)):
        if(assigned_clusters[i] == asdf):
            print(target[i])

CLUST: 0 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
island
closely
roads
boats
towns
waters
prison
opposed
authorities
districts
banks
presents
spots
falls
questioned
assessment
rhythm
preparing
exhibition
auto
pertinent
passengers
trap
assessors
1949
situated
movable
CLUST: 1 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
school
college
university
students
schools
student
interested
active
studied
jewish
faculty
teachers
teaching
newspaper
highest
courses
professor
academic
attend
institute
shooting
varying
magazine
candidates
profession
football
harvard
thursday
campus
mainly
graduate
dartmouth
players
helpful
carleton
pupils
pittsburgh
weekly
portland
managers
publications
furnished
attending
participate
mathematics
listeners
supervision
CLUST: 2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
speak
remain
otherwise
fully
enter
aside
save
possibly
understood
lose
avoid
handle
operate
deny
referred
confusion
replaced
describe
afford
destroyed
assured
fail
admit
conclusions
wear
realistic
alternative
swift
successful

welcome
taught
holy
surely
soul
fought
lucy
wisdom
heaven
believes
unlike
queen
joy
grace
innocent
belong
sympathy
virgin
tells
tears
dying
gift
romantic
lieutenant
eternal
formerly
gentle
overcome
kingdom
torn
mothers
inspired
cancer
souls
relatives
belongs
killer
john's
glory
poverty
motives
mercy
injured
creatures
paula
loves
lover
guards
dreamed
CLUST: 42 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
design
model
practical
gradually
instrument
weakness
satisfactory
furthermore
turns
release
occasional
error
instructions
accurate
remote
missiles
shakespeare
dates
talents
passes
vivid
impressions
economical
delay
CLUST: 43 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
people
group
children
church
members
country
whose
class
women
private
including
england
throughout
organization
movement
groups
persons
negro
professional
relations
churches
americans
parents
citizens
classes
catholic
offered
youth
traditional
mission
formed
expressed
individuals
village
fashion
jobs
vast
friendly
european
acting
setting
ge

tent
cafe
CLUST: 91 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
french
twice
slow
dozen
surprise
moments
silent
cup
liquor
palace
plot
cash
fish
beer
quarter
bird
woodruff
released
mysterious
helva
card
mexican
sheep
surrounded
nest
happily
florida
toast
ham
CLUST: 92 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
one
would
could
time
like
man
even
much
way
little
good
never
us
old
without
place
thought
always
something
enough
better
told
nothing
called
look
asked
knew
young
things
felt
ever
thing
seemed
want
mind
done
kind
name
quite
seen
really
words
word
probably
real
miss
reason
wife
wanted
woman
sometimes
girl
mother
feel
child
hard
says
play
believe
mean
idea
gone
alone
longer
everything
except
live
hope
taking
read
person
dead
lost
heart
feeling
tried
getting
trying
picture
friends
fine
story
answer
thinking
deal
girls
bad
reading
anyone
trouble
note
beautiful
truth
easy
middle
learned
poor
wish
none
hardly
exactly
playing
bit
meant
sir
goes
patient
strange
secret
looks
fair
somehow
ought
helped
clo